In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Multiplexores Cúanticos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [ ]:
from qat.core.console import display
from qat.lang.AQASM import Program, H

In [ ]:
def GetResults(circuit):
    #Create a Job from the circuit
    job = circuit.to_job()
    #Import and create the linear algebra simulator
    from qat.qpus import LinAlg
    linalgqpu = LinAlg()
    #Submit the job to the simulator LinAlg and get the results
    result = linalgqpu.submit(job)
    QP = []
    States = []
    QA = []
    #Print the results
    for sample in result:
        #print("State %s probability %s amplitude %s" % (sample.state, sample.probability, sample.amplitude))
        QP.append(sample.probability)
        States.append(str(sample.state))
        QA.append(sample.amplitude)
    QP = pd.Series(QP, name='Probability')  
    States = pd.Series(States, name='States')  
    QA = pd.Series(QA, name='Amplitude') 
    pdf = pd.concat([States, QP, QA], axis=1)
    return pdf, circuit    
    

## 1. Rotaciones Controladas

Para realizar la carga de una función de probabilidad en un sistema cuántico la operación base es la rotación controlada por un estado previo. 

Vamos a hacer una prueba rápida 

In [ ]:
%load_ext qat.core.magic

In [ ]:
#Cargo mi paquete
sys.path.append('/home/gferro/PhaseAmplification/')
sys.path.append('/home/cesga/gferro/QLM_Backup/Code/PhaseAmplification')

In [ ]:
sys.path.append('/home/gferro/ProgramasDefinitivos/')
sys.path.append('/home/cesga/gferro/QLM_Backup/Code/ProgramasDefinitivos')
from expectation_module import get_histogram

In [ ]:
#Funcion de probabilidad que quiero cargar
def p(x):
    return x*x

In [ ]:
#Rotacion controlada de varios thetas 
def ScRs(Thetas):
    from qat.lang.AQASM import QRoutine
    qrout = QRoutine()
    #Numero de qbits para controlar las Thetas
    NumberOfQbits = int(np.log2(len(Thetas)))
    #Controlling qbits
    qcontrol = qrout.new_wires(NumberOfQbits)
    #Additional qbit where Rotation should be applied
    qtarget = qrout.new_wires(1)
    
    for j, theta in enumerate(Thetas):
        qprog.apply(CRBS_gate(NumberOfQbits, j, theta), qcontrol+qtarget)
    return qrout

Lo primero que voy a hacer es generar un circuito cuántico de dos qbits. El primer qbit controlará la rotación controlada a aplicar sobre el segundo qbit. Como la rotación controlada sobre el segundo qbit dependerá del estado en el que esté el primero lo que haré será poner el primer qbit en una superposición equiprobable de los estados $|0\rangle$ y $|1\rangle$ usando una puerta Haddamard

In [ ]:
nqbits = 2

In [ ]:
qprog = Program()
qbits = qprog.qalloc(nqbits)
for i in range(nqbits-1):
    qprog.apply(H, qbits[i])


In [ ]:
circuit = qprog.to_circ()
%qatdisplay circuit --depth 2

Lo que quiero es aplicar una operación:

$$\hat{U}|q_0q_1\rangle= \{ \begin{array}{ll}
      \hat{R}_y(\theta_0)|q_1\rangle  & if \;|q_0\rangle = |0\rangle \\
      \hat{R}_y(\theta_1)|q_1\rangle  & if \;|q_0\rangle = |1\rangle \\
\end{array}$$

Es decir aplicar una rotacion controlada sobre $|q_1\rangle$ controlada por el **estado** $|q_0\rangle$.

Los angulos $\theta_i$ los calculoen la siguiente celda. Están sacados de la carga de probabilidades pero podrían ser dos ángulos aleatorios.


In [ ]:
a = 0.
b = 1.
nbins = 2**nqbits
centers, probs = get_histogram(p, a, b, nbins)

#Ahora calculo los dos ángulos
m=1
n_parts = 2**(m+1)
edges = np.array([a+(b-a)*(i)/n_parts for i in range(n_parts+1)])
p_zones = np.array([np.sum(probs[np.logical_and(centers>edges[i],centers<edges[i+1])]) for i in range(n_parts)])
p_left = p_zones[[2*j for j in range(n_parts//2)]]
p_tot = p_left + p_zones[[2*j+1 for j in range(n_parts//2)]]
thetas = np.arccos(np.sqrt(p_left/p_tot))

In [ ]:
print('Thetas: {}'.format(thetas))

In [ ]:
from dataloading_module import CRBS_gate

In [ ]:
qprog = Program()
qbits = qprog.qalloc(nqbits)
for i in range(nqbits-1):
    qprog.apply(H, qbits[i])
qprog.apply(ScRs(2.0*thetas), qbits)

In [ ]:
circuitRC = qprog.to_circ()
%qatdisplay circuitRC --depth 2

In [ ]:
pdfRC, _ = GetResults(circuitRC)

In [ ]:
pdfRC

El problema fundamental de esta aproximación es el uso de Rotaciones Controladas que en general suelen ser operaciones difíciles de ejecutar en un ordenador cuántico ...

Además como para cada rotación preparo el estado que le corresponde meto muchas puertas X que posiblemente sean innecesarias ...

## 2.Quantum Multiplexors

Una forma de implementar rotaciones controladas por estados de forma mucho más eficiente es usando multiplexores cuanticos. En este caso se aplican rotaciones completas de ángulos sobre un qbit y se intercalan con operaciones **c-Not**. Esto genera circuitos menos aparatosos donde la complejidad la dan directamente las puertas **c-Not**. El problema base de los multiplexores es que las rotaciones no son directas hay que aplicar combinaciones inteligentes de los ángulos que se quieran rotar. 

Vamos a intentar usar el código de Juan para aplicar estas rotaciones

In [ ]:
#Nos traemos el multiplexor
from expectation_module import multiplexor_RY_m

In [ ]:
qprog = Program()
qbits = qprog.qalloc(nqbits)
for i in range(nqbits-1):
    qprog.apply(H, qbits[i])
multiplexor_RY_m(qprog, qbits, thetas, m, m)

In [ ]:
thetas

In [ ]:
thetas

In [ ]:
[thetas[i] for i in range(0, len(thetas),2)]

In [ ]:
[thetas[i] for i in range(1, len(thetas),2)]

In [ ]:
circuitQM = qprog.to_circ()
%qatdisplay circuitQM --depth 1

In [ ]:
pdfQM,_ = GetResults(circuitQM)

In [ ]:
pdfQM

In [ ]:
def Multiplexor(theta, controlbits):
    #El multiplexor base siempre son 2 qbits
    from qat.lang.AQASM import QRoutine, CNOT, RY
    qrout = QRoutine()
    #Cuantos bits los controlan:
    
    if m>1:
        qbits = qrout.new_wires(controlbits+1)
        thetas4Left = for i in range()
        thetas4Right = 
        pass
    else:
        #Un qbit de control
        qbits = qrout.new_wires(2)
        #Rotaciones. Solo hay dos
        thetaLeft = (theta[0]+theta[1])/2.0
        tethaRight = (theta[0]-theta[1])/2.0
        #Rotacion Izquierda
        qrout.apply(RY(thetaLeft), qbits[1])
        #Puerta c-NOT
        qrout.apply(CNOT, qbits[0], qbits[1])
        #Rotación Derecha
        qrout.apply(RY(tethaRight), qbits[1])
        #rout.apply(CNOT, qbits[0], qbits[1])
    return qrout

In [ ]:
qprog = Program()
qbits = qprog.qalloc(2)
for i in range(2-1):
    qprog.apply(H, qbits[i])
qprog.apply(Multiplexor(2.0*thetas, 1), qbits)

In [ ]:
circuitQMZ = qprog.to_circ()
%qatdisplay circuitQMZ --depth 1

In [ ]:
pdfQMZ,_ = GetResults(circuitQMZ)

In [ ]:
pdfQMZ

In [ ]:
    #Numero de qbits para controlar las Thetas
    NumberOfQbits = int(np.log2(len(Thetas)))
    #Controlling qbits
    qcontrol = qrout.new_wires(NumberOfQbits)
    #Additional qbit where Rotation should be applied
    qtarget = qrout.new_wires(1)
    
    for j, theta in enumerate(Thetas):
        qprog.apply(CRBS_gate(NumberOfQbits, j, theta), qcontrol+qtarget)
    return qrout

In [ ]:
sum(thetas)

In [ ]:
pdfQM,_ = GetResults(circuitQM)

In [ ]:
pdfQM